<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Reinforcement_Learning_with_Proximal_Policy_Optimization_(PPO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gymnasium stable-baselines3

In [ ]:
pip install 'shimmy>=2.0'

In [ ]:
import gymnasium as gym  # Import gymnasium instead of gym
from stable_baselines3 import PPO

# Create the environment using gymnasium
env = gym.make("CartPole-v1")

# Instantiate the PPO agent
model = PPO("MlpPolicy", env, verbose=1)

# Train the agent
model.learn(total_timesteps=10000)

# Save the trained model
model.save("ppo_cartpole")
print("Model saved as ppo_cartpole")

# Load the trained model for evaluation
model = PPO.load("ppo_cartpole", env)

# Evaluate the trained agent with rendering and reward logging
obs, info = env.reset()  # Gymnasium's reset method returns (obs, info)
total_reward = 0

print("\nEvaluating the trained agent...")
for step in range(1000):
    env.render()  # Render the environment to visualize agent behavior
    action, _ = model.predict(obs)  # Predict the next action based on observation
    obs, reward, done, truncated, info = env.step(action)  # Gymnasium's step includes "truncated"
    total_reward += reward
    if done or truncated:  # Reset environment on episode end
        print(f"Episode Reward: {total_reward}")
        total_reward = 0
        obs, info = env.reset()  # Reset also returns (obs, info)

env.close()  # Close the rendering window
print("Evaluation complete!")